In [1]:
from openai import OpenAI
import pandas as pd
from datasets import Dataset
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

c:\Users\phku0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
client = OpenAI(api_key="sk-xxxxxxxxxxxxxxxxx", base_url="https://api.deepseek.com")

In [ ]:
system_message = """
You are a financial analyst extracting structured data from news articles. 
Output JSON format with:
- ticker_sentiments: list of {ticker, sentiment(-1 to 1)}
- sector_sentiments: list of {GICS sector, sentiment(-1 to 1)} 
"""

user_prompt = """
Fossil Fuels Initially Needed for AI Boom, Nuclear Expert Says
Bloomberg News | December 12, 2024
Dale Klein, chair of Tokyo Electric Power Co.'s nuclear reform committee and former head of the U.S. Nuclear Regulatory Commission, stated that the rapid rise in AI and the resulting surge in electricity demand will initially be met by fossil fuels, especially natural gas, before nuclear power can catch up.
Klein noted that data center growth in countries like the U.S. and Japan is outpacing the speed at which nuclear plants can be built. While companies like Microsoft and Google are investing in nuclear to meet clean energy needs, widespread deployment will take time.
He emphasized that nuclear energy will be essential to meet long-term demand while reducing greenhouse gas emissions. In the U.S., even with political support for nuclear, gas will fill the gap temporarily. In Japan, where nuclear remains controversial post-Fukushima, electricity demand could rise again due to new data centers and semiconductor facilities, potentially reigniting the need for nuclear power.
"""

messages = [
  {"role": "system", "content": system_message},
  {"role": "user", "content": user_prompt}
]

response = client.chat.completions.create(
  model="deepseek-chat",
  messages=messages,
  response_format={'type': 'json_object'}
)

In [12]:
print(response.choices[0].message.content)

{
  "ticker_sentiments": [
    {
      "ticker": "MSFT",
      "sentiment": 0.5
    },
    {
      "ticker": "GOOGL",
      "sentiment": 0.5
    },
    {
      "ticker": "TEPCO",
      "sentiment": 0.3
    }
  ],
  "sector_sentiments": [
    {
      "GICS sector": "Energy",
      "sentiment": 0.2
    },
    {
      "GICS sector": "Information Technology",
      "sentiment": 0.6
    },
    {
      "GICS sector": "Utilities",
      "sentiment": 0.4
    }
  ]
}


In [13]:
system_message = """
Analyze the following news article and extract the following information:
1. Determine if the news is related to a merger or acquisition (M&A). Answer "Yes" or "No".  
2. If applicable:  
  - Identify the **acquirer** (company making the purchase).  
  - Identify the **target** (company being acquired).  
  - Extract the **target share price** offered.  
  - Extract the **valuation** (total transaction value).  
3. List all **people names mentioned** in the article.  

Return the output in JSON format with keys:  
- `is_related_to_ma` (boolean)  
- `acquirer` (string or N/A)  
- `target` (string or N/A)  
- `target_share_price` (number or N/A)  
- `valuation` (string or N/A)  
- `people_names` (list of strings or empty list).

If information is unavailable, use "N/A".

Example Output:
{
  "is_related_to_ma": true,
  "acquirer": "Tesla Inc.",
  "target": "SolarCity Corp.",
  "target_share_price": 43,
  "valuation": "1570000000",
  "people_names": ["Elon Musk", "Lyndon Rive"]
}
"""

user_prompt = """
Microsoft to acquire LinkedIn
June 13, 2016
Microsoft Corp. (Nasdaq: MSFT) and LinkedIn Corporation (NYSE: LNKD) on Monday announced they have entered into a definitive agreement under which Microsoft will acquire LinkedIn for $196 per share in an all-cash transaction valued at $26.2 billion, inclusive of LinkedIn's net cash.
LinkedIn will retain its distinct brand, culture and independence. Jeff Weiner will remain CEO of LinkedIn, reporting to Satya Nadella, CEO of Microsoft.
Reid Hoffman, chairman of the board, co-founder and controlling shareholder of LinkedIn, and Weiner both fully support this transaction.
The transaction is expected to close this calendar year.
"""

messages = [
  {"role": "system", "content": system_message},
  {"role": "user", "content": user_prompt}
]

response = client.chat.completions.create(
  model="deepseek-chat",
  messages=messages,
  response_format={'type': 'json_object'}
)

In [14]:
print(response.choices[0].message.content)

{
  "is_related_to_ma": true,
  "acquirer": "Microsoft Corp.",
  "target": "LinkedIn Corporation",
  "target_share_price": 196,
  "valuation": "26.2 billion",
  "people_names": ["Jeff Weiner", "Satya Nadella", "Reid Hoffman"]
}
